In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime as dt

from get_data import Database as db

In [2]:
_y = []
_y_hat = []
_index = []
# max = data.shape[0] - reg_period - vol_period
max = 100

In [3]:
db = db(60, 12)

In [5]:
pd.Series([db.vol['low'][i][1] for i in range(db.hist.shape[0])]).mean()

0.009359396210925576

In [6]:
pd.Series([db.vol['mid'][i][1] for i in range(db.hist.shape[0])]).mean()

0.011701874909992981

In [7]:
pd.Series([db.vol['high'][i][1] for i in range(db.hist.shape[0])]).mean()

0.008774564911335967

In [8]:
r = (pd.Series([db.vol['low'][i][1] for i in range(db.hist.shape[0])])
- pd.Series([db.vol['high'][i][1] for i in range(db.hist.shape[0])])).mean()

In [9]:
s = (pd.Series([db.vol['low'][i][1] for i in range(db.hist.shape[0])])
- pd.Series([db.vol['high'][i][1] for i in range(db.hist.shape[0])])).std()

In [10]:
r/s

0.01039221885964785

In [4]:
pd.Series([db.value['low'][i][1] for i in range(db.hist.shape[0])]).mean()

0.018304112614842622

In [20]:
pd.Series([db.value['low'][i][1] for i in range(db.hist.shape[0])]).mean()

0.018304112614842622

In [21]:
pd.Series([db.value['high'][i][1] for i in range(db.hist.shape[0])]).mean()

0.0020534745282525845

In [22]:
pd.Series([db.value['mid'][i][1] for i in range(db.hist.shape[0])]).mean()

0.007982900297749323

In [26]:
r = (pd.Series([db.value['low'][i][1] for i in range(db.hist.shape[0])])
- pd.Series([db.value['high'][i][1] for i in range(db.hist.shape[0])])).mean()


In [27]:
s = (pd.Series([db.value['low'][i][1] for i in range(db.hist.shape[0])])
- pd.Series([db.value['high'][i][1] for i in range(db.hist.shape[0])])).std()


In [28]:
r / s

0.34733244214644554

In [5]:
pd.Series([db.momentum['low'][i][1] for i in range(db.hist.shape[0])]).mean()

-0.012088331580706844

In [4]:
pd.Series([db.momentum['low'][i][1] for i in range(db.hist.shape[0])]).mean()

-0.012088331580706844

In [5]:
pd.Series([db.momentum['mid'][i][1] for i in range(db.hist.shape[0])]).mean()

0.009761718268191706

In [6]:
pd.Series([db.momentum['high'][i][1] for i in range(db.hist.shape[0])]).mean()

0.031187661218004682

In [18]:
r = (pd.Series([db.momentum['high'][i][1] for i in range(db.hist.shape[0])])
- pd.Series([db.momentum['low'][i][1] for i in range(db.hist.shape[0])])).mean()

In [19]:
s = (pd.Series([db.momentum['high'][i][1] for i in range(db.hist.shape[0])])
- pd.Series([db.momentum['low'][i][1] for i in range(db.hist.shape[0])])).std()

In [20]:
r / s

0.7591646725344757

In [ ]:
class FactorModel:

  def __init__(self, vol_period: int, mom_period: int, reg_period: int):
    '''
      vol_period : Volatility Period
      mom_period : Momentum Period
      reg_period : Regression window
    '''
    db = db(vol_period, mom_period)
    self.vol_period = vol_period
    self.reg_period = reg_period

  

  def gen_trades(cash: float, portfolio: pd.DataFrame, new_stocks, pricing_dt):
    ''' 
      cash : remainder cash position
      portfolio : 
        index : Ticker
        quantity : outstanding position
      new_stocks : list of new positions
      pricing_dt : date of the trade
    '''
    portfolio['c1_price'] = portfolio.index.map(db.hist.loc[pricing_dt,
                                                            portfolio.index])
    try:
      nav = (portfolio['quantity'] * portfolio['c1_price']).sum() + cash
    except:
      nav = cash

    final = pd.DataFrame(db.hist.loc[pricing_dt, new_stocks])
    final.columns = ['c2_price']
    final['quantity_f'] = [nav / len(new_stocks)] * final.shape[0]
    final['quantity_f'] /= final['c2_price']
    final['quantity_f'] = final['quantity_f'].map(lambda x: int(x))

    blotter = portfolio.join(final, how='outer').fillna(0)
    try:
      blotter['quantity'] = (blotter['quantity_f'] - blotter['quantity'])
    except:
      blotter['quantity'] = blotter['quantity_f']

    blotter['cost_price'] = blotter[['c1_price', 'c2_price']].max(axis=1)
    blotter = blotter[['quantity', 'cost_price']]
    blotter['date'] = [pricing_dt] * blotter.shape[0]
    blotter = blotter[blotter['quantity'] != 0]
    blotter.index.name = 'tickers'

    new_cash = cash - (blotter['quantity'] * blotter['cost_price']).sum()
    return (new_cash, blotter)